In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns
import numpy as np
import yfinance as yf

In [3]:
msft = yf.Ticker("MSFT")
msft.info

Exception: yfinance failed to decrypt Yahoo data response

## Part 1: exploratory data analysis - The Forbes 2022 list of the top 100 employers
- Who are they?
- Where do they operate?
- How big are they by employee size?

In [ ]:
# The original .csv of the 'Forbes Top 100 Employer Rankings'
# The list has been merged with each company's:
#     a) Overall Glassdoor Rating as at January 2023
#     b) Ticker symbol to enable financial analysis in section two
#     c) City of Headquarters  

filepath = "resources/employers_clean.csv"
employers_df = pd.read_csv(filepath)

In [ ]:
#initial clean of data:
employers_df=employers_df.replace('South Kore','South Korea')
employers_df=employers_df.rename(columns={'RANK':'Forbes Ranking','NAME':'Employer','City':'HQ City','INDUSTRIES':'Sector','EMPLOYEES':'Number of employees'})
employers_df.set_index('Forbes Ranking')

employers_df.head()

In [ ]:
#American and German companies account for more than half of the top 100
countrygroup=employers_df.groupby('Country')
countrygroup=countrygroup['Forbes Ranking'].count().sort_values(ascending=False)
countrygroup

In [ ]:
sectorgroup=employers_df.groupby('Sector')
sectorgroup=sectorgroup['Sector'].count().sort_values(ascending=False)
sectorgroup

In [ ]:
bins=[0, 100000, 250000, 500000, 2000000]
groups=["0 to \n99,999","100,000 to\n249,999", "250,000 to\n499,999","500,000+" ]

# employers_df['Number of employees']=employers_df['Number of employees'].str.replace(',','')
employers_df['Number of employees']=employers_df['Number of employees'].astype(int)
employers_df['company_size_bins'] = pd.cut(employers_df['Number of employees'],bins,labels=groups,include_lowest=True)

In [ ]:
companysize=employers_df['company_size_bins']
plt.hist(np.sort(companysize))
plt.xlabel("Company size", fontsize=16)
plt.ylabel("Frequency", fontsize=16)
plt.title("Number of companies in Top 100 \nby workforce size",fontsize=16)
plt.show()

## Part 2: Financial analysis of listed companies within Forbes list:
- Research question: do good employers make good investments?
- How do these companies stack up as investments on metrics such as: 12 month stock performance; return on equity, mean analysts recommendations.
- Yfinance wrapper library used to make call to the Yahoo Finance endpoint. Provides the abovementioned metrics within a dictionary through the method 'Ticker.info()'

In [ ]:
employers_financials=employers_df[['Forbes Ranking', 'Employer', 'TICKER','Sector','Country','Glassdoor Rating']]
employers_financials.set_index('Forbes Ranking')

In [ ]:
#This removes the companies without a ticker code
employers_reduced = employers_financials[employers_financials["TICKER"] != 'none'].reset_index(drop=True)


In [ ]:
#This adds new columns for the data that will be brought in using Yfinance
employers_reduced['1 year stock performance']=""
employers_reduced['return on equity']=""
employers_reduced['Analyst Recommendation Mean']=""
employers_reduced

In [ ]:
#This sets up a loop through the rows in the table and searchs each company on its ticker code
#The relevant information is added to the empty columns.
#It also prints the company information to the screen.
#If any of the data is missing, the loop returns "No data found"

print('Finding stock data..')
print("-"*20)
for index,row in employers_reduced.iterrows():
    company=employers_reduced.loc[index,'Employer']
    ticker=employers_reduced.loc[index,'TICKER']
    stock_data=yf.Ticker(ticker)
    try:
        employers_reduced.loc[index, "1 year stock performance"]=round(stock_data.info["52WeekChange"]*100,2)
        employers_reduced.loc[index, "return on equity"]=round(stock_data.info["returnOnEquity"]*100,2)
        employers_reduced.loc[index, "Analyst Recommendation Mean"]=stock_data.info['recommendationMean']
        
    except:
        employers_reduced.loc[index, "1 year stock performance"]="No data found"
        employers_reduced.loc[index, "return on equity"]="No data found"
        employers_reduced.loc[index, "Analyst Recommendation Mean"]="No data found"
        
        
    print(f"{company} ({ticker}): {employers_reduced.loc[index, '1 year stock performance']}% 1yr stock performance; {employers_reduced.loc[index, 'return on equity']}% RoE; Mean Analyst Rec: {employers_reduced.loc[index, 'Analyst Recommendation Mean']}")
    print("-"*20)

employers_reduced